In [1]:
%load_ext autoreload
%autoreload 2
import sys
import os
import glob
import random
from collections import defaultdict
from pathlib import Path

from IPython.display import display

import pandas as pd
import dask

from tqdm import tqdm
import jax
jax.config.update('jax_platform_name', 'cpu')
# jax.config.update('jax_log_compiles', True)
# jax.config.update("jax_debug_nans", True)
# jax.config.update("jax_enable_x64", True)

In [2]:


sys.path.append("../..")

from lib import utils as U
from lib.ehr import load_dataset, load_dataset_scheme, load_dataset_config, Dataset
from lib.ehr.interface import Patients, InterfaceConfig
from lib.ehr.concepts import CPRDDemographicVectorConfig, DemographicVectorConfig


In [3]:
import logging
logging.root.level = logging.INFO

In [4]:
tag = 'CPRD'
PATH = f'{os.environ.get("HOME")}/GP/ehr-data/cprd-data/DUMMY_DATA.csv'
# PATH = f'{HOME}/Documents/DS211/users/tb1009/DATA/PAT_COHORT/ICENODE_SUBSET_1000.csv'
sample = None
cache =  f'cached_inteface/patients_{tag}_{sample or ""}'
dataset_config = load_dataset_config(tag, 
                                     sample=sample,
                                     path=PATH)


##### Possible Interface Scheme Configurations

In [5]:
import json
dataset_scheme = load_dataset_scheme(tag)
interface_schem_options = dataset_scheme.supported_target_scheme_options
print(json.dumps(interface_schem_options, sort_keys=True, indent=4))

{
    "dx": [
        "DxLTC9809FlatMedcodes",
        "DxLTC212FlatCodes"
    ],
    "ethnicity": [
        "CPRDEthnicity16",
        "CPRDEthnicity5"
    ],
    "gender": [
        "CPRDGender"
    ],
    "imd": [
        "CPRDIMDCategorical"
    ],
    "outcome": [
        "dx_cprd_ltc212",
        "dx_cprd_ltc9809"
    ]
}


In [6]:
# Demographic vector attributes
demographic_vector_conf = CPRDDemographicVectorConfig(age=True,
                                                      gender=True,
                                                      ethnicity=True,
                                                      imd=True)

interface_scheme = dataset_scheme.make_target_scheme_config(dx='DxLTC9809FlatMedcodes',
                                                            #outcome='dx_cprd_ltc212',
                                                            outcome='dx_cprd_ltc9809',
                                                            ethnicity='CPRDEthnicity5')
interface_config = InterfaceConfig(scheme=interface_scheme,
                                   dataset_scheme=dataset_scheme,
                                   demographic_vector=demographic_vector_conf,
                                   cache=cache)

In [7]:
from lib.ml import (ICENODE, ICENODEConfig, GRU, GRUConfig, RETAIN, RETAINConfig,
                    OutpatientEmbeddingConfig,  SplitConfig,
                    Trainer, TrainerConfig, TrainerReporting, OptimizerConfig, WarmupConfig, ReportingConfig)
from lib.metric import  (CodeAUC, UntilFirstCodeAUC, AdmissionAUC, CodeLevelMetricConfig, MetricLevelsConfig,
                         LossMetricConfig,
                      CodeGroupTopAlarmAccuracy, LossMetric, ObsCodeLevelLossMetric, CodeGroupTopAlarmAccuracyConfig)
from lib.ml import Experiment, ExperimentConfig, SplitConfig

import jax.random as jrandom

In [8]:
# emb_dims = OutpatientEmbeddingConfig(dx=30, demo=15)
# model_config = ICENODEConfig(mem=15, emb=emb_dims)
# model_classname = ICENODE.__name__

In [9]:
# emb_dims = OutpatientEmbeddingConfig(dx=30, demo=15)
# model_config = GRUConfig(emb=emb_dims)
# model_classname = GRU.__name__

In [10]:
emb_dims = OutpatientEmbeddingConfig(dx=30, demo=15)
model_config = RETAINConfig(mem_a=15, mem_b=15, emb=emb_dims)
model_classname = RETAIN.__name__

In [11]:
trainer_config = TrainerConfig(optimizer=OptimizerConfig(opt='adam', lr=1e-3),
                          epochs=80,
                          batch_size=128,
                          dx_loss='balanced_focal_bce',
                          obs_loss='mse',
                          lead_loss='mse')

warmup = WarmupConfig(epochs=0.1, 
                      batch_size=8,
                      opt='adam', lr=1e-3, 
                      decay_rate=0.5)




In [12]:
dx_loss = ["softmax_bce", "balanced_focal_softmax_bce", "balanced_focal_bce",
          "allpairs_exp_rank", "allpairs_hard_rank", "allpairs_sigmoid_rank"]
obs_loss =  ["mse", "mae", "rms"]
                
metrics_conf = [
    (CodeAUC, CodeLevelMetricConfig(aggregate_level=True, code_level=True)),
    (AdmissionAUC, MetricLevelsConfig(admission=False, aggregate=True, subject_aggregate=False)),
    (CodeGroupTopAlarmAccuracy, CodeGroupTopAlarmAccuracyConfig(n_partitions=5, top_k_list=[3, 5, 10, 15, 20])),
    (LossMetric, LossMetricConfig(dx_loss=dx_loss))
]
metrics_conf = [m.export_module_class(c) for m, c in metrics_conf]

In [13]:
reporting_conf = ReportingConfig(output_dir='icenode',
                                 console=True,
                                 model_stats=False,
                                 parameter_snapshots=True,
                                 config_json=True)

In [14]:
expt_config = ExperimentConfig(dataset=dataset_config,
                              interface=interface_config,
                              split=SplitConfig(train=0.8, val=0.1, test=0.1, balanced='admissions'),
                              trainer=trainer_config,
                              metrics=metrics_conf,
                              reporting=reporting_conf,
                              model=model_config,
                              model_classname=model_classname,
                              n_evals=100,
                              continue_training=True,
                              warmup=None,
                              reg_hyperparams=None)

In [15]:
experiment = Experiment(expt_config)

In [16]:
expt_config

ExperimentConfig(
  dataset=DatasetConfig(
    path='/home/asem/GP/ehr-data/cprd-data/DUMMY_DATA.csv',
    scheme=CPRDDatasetSchemeConfig(
      dx='DxLTC9809FlatMedcodes',
      ethnicity='CPRDEthnicity16',
      gender='CPRDGender',
      outcome=None,
      imd='CPRDIMDCategorical'
    ),
    scheme_classname='CPRDDatasetScheme',
    colname={
      'adm':
      {
        'age_at_dischtime':
        'age_list',
        'code':
        'medcode_list',
        'dischtime':
        'year_month_list',
        'ethnicity':
        'ethnicity',
        'gender':
        'gender',
        'imd_decile':
        'imd_decile',
        'subject_id':
        'patid'
      }
    },
    files={},
    sample=None,
    meta_fpath='',
    tag='CPRD',
    overlapping_admissions='merge'
  ),
  interface=InterfaceConfig(
    demographic_vector=CPRDDemographicVectorConfig(
      gender=True,
      age=True,
      ethnicity=True,
      imd=True
    ),
    leading_observable=None,
    scheme=CPRDDatasetSc

In [17]:
result = experiment.run()

INFO:root:Loading cached subjects.
INFO:jax._src.xla_bridge:Unable to initialize backend 'rocm': NOT_FOUND: Could not find registered platform with name: "rocm". Available platform names are: CUDA Interpreter
INFO:jax._src.xla_bridge:Unable to initialize backend 'tpu': module 'jaxlib.xla_extension' has no attribute 'get_tpu_client'
INFO:root:Continuing training from step 0
INFO:root:HPs: TrainerConfig(
  optimizer=OptimizerConfig(
    opt='adam',
    lr=0.001,
    decay_rate=None,
    reverse_schedule=False
  ),
  epochs=80,
  batch_size=128,
  dx_loss='balanced_focal_bce',
  obs_loss='mse',
  lead_loss='mse'
)


Loading to device:   0%|          | 0/19 [00:00<?, ?subject/s]

  0%|          | 0/80 [00:00<?, ?Epoch/s]

  0%|          | 0/1 [00:00<?, ?Batch/s]

Loading to device:   0%|          | 0/144 [00:00<?, ?subject/s]

Embedding:   0%|          | 0/144 [00:00<?, ?subject/s]

  0%|          | 0.00/224489.00 [00:00<?, ?longitudinal-days/s]

TypeError: mul got incompatible shapes for broadcasting: (30,), (45,).

###### 